# Thanks, nine

In [1]:
import json
import requests
import pandas as pd
from tqdm import tqdm

In [2]:
df = pd.read_csv('master_cleansing.csv', index_col = 0)
df

,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2
no,,,,,,,,
7658,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,동대문구,회기동
1551,중형승합,2018-01-23 00:17:00,2018-01-23 00:50:37,2018-01-23 01:19:22,중랑구,면목제7동,노원구,상계1동
5804,중형승합,2018-01-23 00:30:00,2018-01-23 00:17:06,2018-01-23 00:17:39,성북구,정릉제3동,강북구,수유제3동
7650,중형승합,2018-01-23 00:47:00,2018-01-23 00:51:19,2018-01-23 01:09:33,구로구,고척제2동,관악구,은천동
1061,중형승합,2018-01-23 01:08:00,2018-01-23 01:44:17,2018-01-23 02:06:53,동대문구,회기동,성북구,장위제2동
...,...,...,...,...,...,...,...,...
8579,대형승용,2022-06-24 07:00:00,2022-06-24 07:12:38,2022-06-24 07:55:24,성북구,돈암제2동,강남구,대치4동
8006,중형승합,2022-06-24 07:00:00,2022-06-24 07:09:10,2022-06-24 07:25:35,은평구,구산동,은평구,구산동
8261,중형승합,2022-06-24 07:00:00,2022-06-24 07:04:27,2022-06-24 07:25:35,노원구,상계8동,영등포구,영등포동


# Geocoding하기

In [3]:
## 이 부분을 수정하세요

# ID, Key 입력
ID = "069o3ky18n"
Key = "AKutx4Oy4jjqjAy4iRaxziaYgyGW8NmcKWP7u16c"

# 검색할 주소 리스트로 입력
startpos = list(set(df['startpos2']))
endpos = list(set(df['endpos2']))

In [4]:
## 여기는 수정하지 마세요

# URL 설정
url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"

# 헤더 설정
headers = {'X-NCP-APIGW-API-KEY-ID' : ID, 'X-NCP-APIGW-API-KEY' : Key}

# 기타 설정
pbar_start = tqdm(total=len(startpos))
pbar_end = tqdm(total=len(endpos))

  0%|          | 0/557 [00:00<?, ?it/s]

In [5]:
# 스타트포스의 고유값들을 지오코딩해서 lat,lng에 넣기
start_lat = []
start_lng = []
for i in range(len(startpos)):
    params = {'query' : startpos[i]}
    json_data = requests.get(url, headers=headers, params=params).json()
    try:
        start_lat.append(json_data['addresses'][0]['x'])
        start_lng.append(json_data['addresses'][0]['y'])
    except Exception as e:
        start_lat.append(0)
        start_lng.append(0)
    pbar_start.update(1)
pbar_start.close()

100%|██████████| 557/557 [02:13<00:00,  4.16it/s]


In [6]:
# 엔드포스의 고유값들을 지오코딩해서 lat,lng에 넣기
end_lat = []
end_lng = []
for i in range(len(endpos)):
    params = {'query' : endpos[i]}
    json_data = requests.get(url, headers=headers, params=params).json()
    try:
        end_lat.append(json_data['addresses'][0]['x'])
        end_lng.append(json_data['addresses'][0]['y'])
    except Exception as e:
        end_lat.append(0)
        end_lng.append(0)
    pbar_end.update(1)
pbar_end.close()

100%|██████████| 691/691 [05:07<00:00,  2.25it/s]


In [16]:
# 스타트포스, 엔드포스 고유값 데이터프레임화 (0,0인 위경도가 포함되어 있음)
# before, after를 만든 이유? 0,0인 위경도의 이름을 구별하기 위해 찾아내야 함
df_start = pd.DataFrame(zip(startpos, startpos, start_lat, start_lng), columns=['before', 'after', 'start_lat', 'start_lng'])
df_end = pd.DataFrame(zip(endpos, endpos, end_lat, end_lng), columns=['before', 'after', 'end_lat', 'end_lng'])

In [17]:
df_start_replace = df_start[df_start['start_lat']==0]
df_end_replace = df_end[df_end['end_lat']==0]

In [18]:
# 바뀌어야 하는 행정동 이름 딕셔너리화
repl = {"제기제2동" : "제기동",
        "이문제3동" : "이문3동",
        "상도제1동" : "상도1동",
        "가능1동" : "가능동",
        "신당제4동" : "신당4동",
        "신당제3동" : "신당3동",
        "장안제4동" : "장안2동",
        "신당제6동" : "신당6동",
        "신당제1동" : "신당1동",
        "전농제3동" : "전농3동",
        "김포1동" : "김포본동",
        "면목제3.8동" : "면목3.8동",
        "장안제3동" : "장안1동",
        "신당제2동" : "신당2동",
        "명륜3가동" : "명륜3가",
        "답십리제4동" : "답십리4동",
        "제기제1동" : "제기동",
        "답십리제3동" : "답십리3동",
        "김포2동" : "장기본동",
        "의정부3동" : "의정부1동",
        "가능3동" : "흥선동",
        "검단4동" : "당하동",
        "검단3동" : "원당동",
        "신도동" : "삼송동",
        "교하읍" : "교하동",
        "가능2동" : "흥선동"
        }

In [19]:
# after 값에 행정동 바꾸기
for i in df_start_replace.index:
    df_start_replace.loc[i, 'after'] = repl.get(df_start_replace.loc[i, 'before'])
for i in df_end_replace.index:
    df_end_replace.loc[i, 'after'] = repl.get(df_end_replace.loc[i, 'before'])

/var/folders/yl/v8cdbc9j04n78s5y2d8dhqpr0000gn/T/ipykernel_31240/1111201305.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_start_replace.loc[i, 'after'] = repl.get(df_start_replace.loc[i, 'before'])
/var/folders/yl/v8cdbc9j04n78s5y2d8dhqpr0000gn/T/ipykernel_31240/1111201305.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_end_replace.loc[i, 'after'] = repl.get(df_end_replace.loc[i, 'before'])


In [20]:
# 인덱스 임시 저장해놓기
df_start_replace = df_start_replace.reset_index()
df_end_replace = df_end_replace.reset_index()

In [21]:
# 스타트포스의 0,0 위경도를 변경된 이름으로 재검색
be_dum_params = df_start_replace['before']
af_dum_params = df_start_replace['after']
for i in range(len(df_start_replace)):
    params = {'query' : af_dum_params[i]}
    dummy = requests.get(url, headers=headers, params=params).json()
    dummy_lat = dummy['addresses'][0]['x']
    dummy_lng = dummy['addresses'][0]['y']
    df_start_replace.loc[i, 'start_lat'] = dummy_lat
    df_start_replace.loc[i, 'start_lng'] = dummy_lng
df_start_replace

,index,before,after,start_lat,start_lng
0,72,장안제3동,장안1동,127.066375,37.567842
1,76,면목제3.8동,면목3.8동,127.0880603,37.5829681
2,143,제기제1동,제기동,127.0378551,37.5831612
3,173,답십리제3동,답십리3동,127.0512678,37.5717963
4,176,이문제3동,이문3동,127.065075,37.6033571
5,200,신당제2동,신당2동,127.0082322,37.5542861
6,228,김포2동,장기본동,126.6716999,37.6523061
7,240,장안제4동,장안2동,127.0706,37.578483
8,300,가능1동,가능동,127.0384877,37.7505409
9,309,김포1동,김포본동,126.7057,37.6275987


In [22]:
# 엔드포스의 0,0 위경도를 변경된 이름으로 재검색
be_dum_params = df_end_replace['before']
af_dum_params = df_end_replace['after']
for i in range(len(df_end_replace)):
    params = {'query' : af_dum_params[i]}
    dummy = requests.get(url, headers=headers, params=params).json()
    dummy_lat = dummy['addresses'][0]['x']
    dummy_lng = dummy['addresses'][0]['y']
    df_end_replace.loc[i, 'end_lat'] = dummy_lat
    df_end_replace.loc[i, 'end_lng'] = dummy_lng
df_end_replace

,index,before,after,end_lat,end_lng
0,81,의정부3동,의정부1동,127.0475771,37.7464356
1,88,장안제3동,장안1동,127.066375,37.567842
2,95,면목제3.8동,면목3.8동,127.0880603,37.5829681
3,161,가능3동,흥선동,127.0306961,37.7463446
4,181,제기제1동,제기동,127.0378551,37.5831612
5,220,답십리제3동,답십리3동,127.0512678,37.5717963
6,224,이문제3동,이문3동,127.065075,37.6033571
7,251,신당제2동,신당2동,127.0082322,37.5542861
8,284,김포2동,장기본동,126.6716999,37.6523061
9,301,장안제4동,장안2동,127.0706,37.578483


In [23]:
# 인덱스 다시 설정하기
df_start_replace.set_index('index')
df_end_replace.set_index('index')

,before,after,end_lat,end_lng
index,,,,
81,의정부3동,의정부1동,127.0475771,37.7464356
88,장안제3동,장안1동,127.066375,37.567842
95,면목제3.8동,면목3.8동,127.0880603,37.5829681
161,가능3동,흥선동,127.0306961,37.7463446
181,제기제1동,제기동,127.0378551,37.5831612
220,답십리제3동,답십리3동,127.0512678,37.5717963
224,이문제3동,이문3동,127.065075,37.6033571
251,신당제2동,신당2동,127.0082322,37.5542861
284,김포2동,장기본동,126.6716999,37.6523061


In [24]:
# 스타트포스 고유값 데이터프레임화 (0,0인 위경도가 변경 완료)
# join할 때 before를 위주로 하면 됨
for i in range(len(df_start_replace)):
    df_start.loc[df_start['before'] == df_start_replace.before[i], 'start_lat'] = df_start_replace.start_lat[i]
    df_start.loc[df_start['before'] == df_start_replace.before[i], 'start_lng'] = df_start_replace.start_lng[i]
for i in range(len(df_end_replace)):
    df_end.loc[df_end['before'] == df_end_replace.before[i], 'end_lat'] = df_end_replace.end_lat[i]
    df_end.loc[df_end['before'] == df_end_replace.before[i], 'end_lng'] = df_end_replace.end_lng[i]

In [25]:
# 스타트포스 조인
df_join_start = pd.merge(df, df_start, left_on='startpos2', right_on='before', how='left')
df_join_start = df_join_start.drop(['before','after'], axis=1)
df_join_start

,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,start_lat,start_lng
0,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,동대문구,회기동,127.0437129,37.6141245
1,중형승합,2018-01-23 00:17:00,2018-01-23 00:50:37,2018-01-23 01:19:22,중랑구,면목제7동,노원구,상계1동,127.0869854,37.5789945
2,중형승합,2018-01-23 00:30:00,2018-01-23 00:17:06,2018-01-23 00:17:39,성북구,정릉제3동,강북구,수유제3동,127.004186,37.6089578
3,중형승합,2018-01-23 00:47:00,2018-01-23 00:51:19,2018-01-23 01:09:33,구로구,고척제2동,관악구,은천동,126.8587122,37.5067055
4,중형승합,2018-01-23 01:08:00,2018-01-23 01:44:17,2018-01-23 02:06:53,동대문구,회기동,성북구,장위제2동,127.0553132,37.5908083
...,...,...,...,...,...,...,...,...,...,...
407548,대형승용,2022-06-24 07:00:00,2022-06-24 07:12:38,2022-06-24 07:55:24,성북구,돈암제2동,강남구,대치4동,127.0105659,37.598039
407549,중형승합,2022-06-24 07:00:00,2022-06-24 07:09:10,2022-06-24 07:25:35,은평구,구산동,은평구,구산동,126.9103893,37.6117455
407550,중형승합,2022-06-24 07:00:00,2022-06-24 07:04:27,2022-06-24 07:25:35,노원구,상계8동,영등포구,영등포동,127.0515874,37.6667854
407551,중형승합,2022-06-24 07:00:00,2022-06-24 08:06:22,2022-06-24 06:55:01,성북구,길음제1동,강북구,번제1동,127.0193925,37.6104856


In [26]:
# 엔드포스 조인
df_join_end = pd.merge(df_join_start, df_end, left_on='endpos2', right_on='before', how='left')
df_join_end = df_join_end.drop(['before','after'], axis=1)
df_join_end

,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,start_lat,start_lng,end_lat,end_lng
0,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,동대문구,회기동,127.0437129,37.6141245,127.0553132,37.5908083
1,중형승합,2018-01-23 00:17:00,2018-01-23 00:50:37,2018-01-23 01:19:22,중랑구,면목제7동,노원구,상계1동,127.0869854,37.5789945,127.0549781,37.6799006
2,중형승합,2018-01-23 00:30:00,2018-01-23 00:17:06,2018-01-23 00:17:39,성북구,정릉제3동,강북구,수유제3동,127.004186,37.6089578,127.023125,37.638715
3,중형승합,2018-01-23 00:47:00,2018-01-23 00:51:19,2018-01-23 01:09:33,구로구,고척제2동,관악구,은천동,126.8587122,37.5067055,126.9424278,37.4853086
4,중형승합,2018-01-23 01:08:00,2018-01-23 01:44:17,2018-01-23 02:06:53,동대문구,회기동,성북구,장위제2동,127.0553132,37.5908083,127.0545206,37.6123899
...,...,...,...,...,...,...,...,...,...,...,...,...
407548,대형승용,2022-06-24 07:00:00,2022-06-24 07:12:38,2022-06-24 07:55:24,성북구,돈암제2동,강남구,대치4동,127.0105659,37.598039,127.0579127,37.4997415
407549,중형승합,2022-06-24 07:00:00,2022-06-24 07:09:10,2022-06-24 07:25:35,은평구,구산동,은평구,구산동,126.9103893,37.6117455,126.9103893,37.6117455
407550,중형승합,2022-06-24 07:00:00,2022-06-24 07:04:27,2022-06-24 07:25:35,노원구,상계8동,영등포구,영등포동,127.0515874,37.6667854,126.9106918,37.5203985
407551,중형승합,2022-06-24 07:00:00,2022-06-24 08:06:22,2022-06-24 06:55:01,성북구,길음제1동,강북구,번제1동,127.0193925,37.6104856,127.0288473,37.6378913


In [29]:
df_join_end.to_csv("master_cleansing_map_after.csv", encoding="utf-8")